In [1]:
import gather_keys_oauth2 as Oauth2
import fitbit
import pandas as pd 
import datetime
import hashlib
import random
import json
from tqdm import tqdm
from datetime import datetime, timedelta

CLIENT_ID='23QSLM'
CLIENT_SECRET='8ed82541d286c938f9b8bc3f4e46da82'

In [2]:
server=Oauth2.OAuth2Server(CLIENT_ID, CLIENT_SECRET)
server.browser_authorize()
ACCESS_TOKEN=str(server.fitbit.client.session.token['access_token'])
REFRESH_TOKEN=str(server.fitbit.client.session.token['refresh_token'])
auth2_client=fitbit.Fitbit(CLIENT_ID,CLIENT_SECRET,oauth2=True,access_token=ACCESS_TOKEN,refresh_token=REFRESH_TOKEN)

[30/Apr/2023:18:10:08] ENGINE Listening for SIGTERM.
[30/Apr/2023:18:10:08] ENGINE Listening for SIGHUP.
[30/Apr/2023:18:10:08] ENGINE Listening for SIGUSR1.
[30/Apr/2023:18:10:08] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[30/Apr/2023:18:10:08] ENGINE Started monitor thread 'Autoreloader'.
[30/Apr/2023:18:10:08] ENGINE Serving on http://127.0.0.1:8080
[30/Apr/2023:18:10:08] ENGINE Bus STARTED


127.0.0.1 - - [30/Apr/2023:18:10:10] "GET /?code=8556567d310d2ce61de85949773422b8e9e56176&state=ODqoKvrcKVfnrzVW6b5soEVsZhvWZ8 HTTP/1.1" 200 122 "" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.4 Safari/605.1.15"
127.0.0.1 - - [30/Apr/2023:18:10:10] "GET /favicon.ico HTTP/1.1" 200 1406 "http://127.0.0.1:8080/?code=8556567d310d2ce61de85949773422b8e9e56176&state=ODqoKvrcKVfnrzVW6b5soEVsZhvWZ8" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.4 Safari/605.1.15"


[30/Apr/2023:18:10:11] ENGINE Bus STOPPING
[30/Apr/2023:18:10:11] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[30/Apr/2023:18:10:11] ENGINE Stopped thread 'Autoreloader'.
[30/Apr/2023:18:10:11] ENGINE Bus STOPPED
[30/Apr/2023:18:10:11] ENGINE Bus EXITING
[30/Apr/2023:18:10:11] ENGINE Bus EXITED
[30/Apr/2023:18:10:11] ENGINE Waiting for child threads to terminate...


In [18]:
def generate_id():
    random_str = str(random.getrandbits(256)).encode('utf-8')
    sha2_hash = hashlib.sha256(random_str).hexdigest()
    return sha2_hash[:32]

### Create Heart Rate Data

In [18]:
startTime = pd.to_datetime('2023-04-01',format='%Y-%m-%d')
endTime = pd.to_datetime('2023-04-28',format='%Y-%m-%d')

allDates = pd.date_range(start=startTime, end = endTime)

pbar = tqdm(total=len(allDates))
documents = []

for date in allDates:

    oneDayData = auth2_client.intraday_time_series('activities/heart',base_date=date,detail_level='15min')

    heartRateZones = oneDayData['activities-heart'][0]['value']['heartRateZones']

    for zone in heartRateZones:
        heartRateZones_dict = {
            'id' : generate_id(),
            'type' : 'heart-rate-zone',
            'data' : {
                'date' : date.date().strftime("%Y-%m-%d")
            }
        }

        for key, value in zone.items():
            heartRateZones_dict['data'][key] = value

        documents.append(heartRateZones_dict)

    heartIntraday = oneDayData['activities-heart-intraday']['dataset']

    for heartrate in heartIntraday:

        # date_obj = datetime.strptime(date, '%Y-%m-%d')
        time_obj = datetime.strptime(heartrate['time'], '%H:%M:%S').time()
        new_date_obj = datetime.combine(date, time_obj) + timedelta(hours=0)

        heartRate_dict = {
            'id' : generate_id(),
            'type' : 'heart-rate',
            'data' : {
                'datetime' : datetime.strftime(new_date_obj, '%Y-%m-%d %H:%M:%S'),
                'value' : heartrate['value']
            }
        }

        documents.append(heartRate_dict)

    pbar.update(1)


pbar.close()

json_str = json.dumps(documents)
with open(f'documents_heart_rate.json', 'w') as f:
    f.write(json_str)

100%|██████████| 28/28 [00:08<00:00,  3.28it/s]


### Create Activities Data

In [71]:
startTime = pd.to_datetime('2023-04-01',format='%Y-%m-%d')
endTime = pd.to_datetime('2023-04-28',format='%Y-%m-%d')

allDates = pd.date_range(start=startTime, end = endTime)

documents = []
activities_df_list = []

for date in allDates:

    oneDayData = auth2_client.activities(date=date)

    activities = oneDayData['activities']

    for activity in activities:

        activity_dict ={
            'id' : generate_id(),
            'type' : 'activity',
            'data' : activity
        }

        activity_dict['data']['date'] = date.date().strftime("%Y-%m-%d")
        documents.append(activity_dict)

        activities_df = pd.DataFrame(activity, index=[0])
        activities_df['date'] = date.date().strftime("%Y-%m-%d")
        activities_df_list.append(activities_df)

json_str = json.dumps(documents)
with open(f'documents_activities.json', 'w') as f:
    f.write(json_str)

activities_df_final = pd.concat(activities_df_list, axis = 0)

activities_df_final.to_csv('activities.csv',index=False)
activities_df_final

HTTPTooManyRequests: Too Many Requests

### Create Distances and Activitis Summary Data

In [62]:
startTime = pd.to_datetime('2023-04-01',format='%Y-%m-%d')
endTime = pd.to_datetime('2023-04-28',format='%Y-%m-%d')

allDates = pd.date_range(start=startTime, end = endTime)


distances_df_list = []
activities_summary_df_list = []

documents = []
for date in allDates:

    oneDayData = auth2_client.activities(date=date)

    # Get distances
    distances = oneDayData['summary']['distances']

    for distance in distances:
        distance_dict = {
            'id' : generate_id(),
            'type' : 'distance',
            'data' : distance
        }

        distance_dict['data']['date'] = date.date().strftime("%Y-%m-%d")

        documents.append(distance_dict)

    distances_df = pd.DataFrame(distances)
    distances_df['date'] = date.date().strftime("%Y-%m-%d")
    distances_df_list.append(distances_df)

    dict_for_df = {}

    for data in ['steps','floors','elevation','lightlyActiveMinutes','fairlyActiveMinutes','veryActiveMinutes']:
        data_value = oneDayData['summary'][data]
        data_dict = {
                'id' : generate_id(),
                'type' : data,
                'data' : {
                    'date' : date.date().strftime("%Y-%m-%d"),
                    'value' : data_value
                }
            }
        documents.append(data_dict)

        dict_for_df[data] = { 'value' : data_value}

    activities_summary = pd.DataFrame(dict_for_df)
    activities_summary.reset_index(drop=True, inplace=True)
    activities_summary['date'] = date.date().strftime("%Y-%m-%d")
    activities_summary_df_list.append(activities_summary)

distances_df_final = pd.concat(distances_df_list, axis = 0)
activities_summary_df_final = pd.concat(activities_summary_df_list, axis = 0)

distances_df_final.to_csv('distances.csv',index=False)
activities_summary_df_final.to_csv('activities_summary.csv',index=False)

json_str = json.dumps(documents)
with open(f'documents_distances.json', 'w') as f:
    f.write(json_str)

In [13]:
date = pd.to_datetime('2023-04-28',format='%Y-%m-%d')
oneDayData = auth2_client.activities(date=date)
print(oneDayData)
distances = oneDayData['activities']['summary']['distances']
for distance in distances:
    

{'activities': [{'activityId': 90013, 'activityParentId': 90013, 'activityParentName': 'Walk', 'calories': 363, 'description': 'Walking less than 2 mph, strolling very slowly', 'duration': 5171000, 'hasActiveZoneMinutes': True, 'hasStartTime': True, 'isFavorite': False, 'lastModified': '2023-04-29T16:06:01.000Z', 'logId': 55439337001, 'name': 'Walk', 'startDate': '2023-04-28', 'startTime': '20:49', 'steps': 6269}], 'goals': {'activeMinutes': 30, 'caloriesOut': 2232, 'distance': 5, 'floors': 10, 'steps': 10000}, 'summary': {'activeScore': -1, 'activityCalories': 1017, 'caloriesBMR': 1330, 'caloriesOut': 2115, 'distances': [{'activity': 'total', 'distance': 4.44}, {'activity': 'tracker', 'distance': 4.44}, {'activity': 'loggedActivities', 'distance': 0}, {'activity': 'veryActive', 'distance': 0.37}, {'activity': 'moderatelyActive', 'distance': 2.05}, {'activity': 'lightlyActive', 'distance': 2.01}, {'activity': 'sedentaryActive', 'distance': 0}], 'elevation': 250, 'fairlyActiveMinutes': 

[{'activityId': 90013,
  'activityParentId': 90013,
  'activityParentName': 'Walk',
  'calories': 363,
  'description': 'Walking less than 2 mph, strolling very slowly',
  'duration': 5171000,
  'hasActiveZoneMinutes': True,
  'hasStartTime': True,
  'isFavorite': False,
  'lastModified': '2023-04-29T16:06:01.000Z',
  'logId': 55439337001,
  'name': 'Walk',
  'startDate': '2023-04-28',
  'startTime': '20:49',
  'steps': 6269}]

In [15]:
df = pd.read_csv('heartRateZones.csv')
df_pivoted = df.pivot(index=['date'], columns='name', values='minutes')
df_pivoted.columns = [col for col in df_pivoted.columns.values]
df_pivoted.reset_index(inplace=True)
df_pivoted

,date,Cardio,Fat Burn,Out of Range,Peak
0,2023-04-01,0,8,1432,0
1,2023-04-02,0,19,1421,0
2,2023-04-03,0,1,1439,0
3,2023-04-04,0,0,1440,0
4,2023-04-05,0,4,1436,0
5,2023-04-06,1,60,1379,0
6,2023-04-07,1,20,1419,0
7,2023-04-08,0,10,1430,0
8,2023-04-09,0,1,1439,0
9,2023-04-10,0,3,1437,0
